In [91]:
# import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings


### x_data , y_data , dataframe 형태
tf.reset_default_graph()

train = pd.read_csv("./data/mnist2/train.csv")
test = pd.read_csv("./data/mnist2/test.csv")


x_data=train.drop("label", axis=1, inplace=False)
train2=train["label"]

 
df1=pd.DataFrame()
arr = np.zeros((10,10), dtype=np.int32)
df1=arr
df2=pd.DataFrame()
for n in range(0,10):
    df1[n,n]=1
    df2[n]=np.arange(10)

tr2=int(len(train2))
for n in range(0,tr2):
    df2[n]=df1[train2[n]]
y_data=pd.DataFrame()
y_data=df2.transpose()


x_data=x_data.values
y_data=y_data.values
### x_data , y_data , dataframe 형태 넘파이 어레이가 공통적으로 쓰임

class CNNModel: 
    
    def __init__(self,sess,name):     

        self.sess=sess
        self.name=name 
        
    def build_graph(self):
        with tf.variable_scope(self.name):
            self.X=tf.placeholder(shape=[None,784], dtype=tf.float32)
            self.Y=tf.placeholder(shape=[None,10], dtype=tf.float32)
            self.keep_rate=tf.placeholder(dtype=tf.float32) 
            
            X_img = tf.reshape(self.X, shape=[-1,28,28,1])

            W1=tf.Variable(tf.random_normal([5,5,1,32],stddev=0.01))

            L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1], padding="SAME")

            L1 = tf.nn.relu(L1)
            p1 = tf.nn.max_pool(L1 , ksize=[1,2,2,1] , strides=[1,2,2,1], padding="SAME")

            L2=tf.layers.conv2d(inputs=p1, filters=64, kernel_size=[5,5] , padding="SAME", strides=1 ,activation=tf.nn.relu)

            L2=tf.layers.max_pooling2d(inputs=L2 , pool_size=[2,2], padding="SAME" , strides=2)

            L2=tf.reshape(L2,shape=[-1,7*7*64])

            W2=tf.get_variable("weight2" , shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
            b2=tf.Variable(tf.random_normal([256]),name="bias2" )

            _layer1 = tf.nn.relu(tf.matmul(L2,W2)+b2 )
            layer1 = tf.nn.dropout(_layer1 , keep_prob=self.keep_rate)


            W3=tf.get_variable("weight3" , shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
            b3=tf.Variable(tf.random_normal([256]),name="bias3" )

            _layer2 = tf.nn.relu(tf.matmul(layer1,W3)+b3 )
            layer2 = tf.nn.dropout(_layer2 , keep_prob=self.keep_rate)


            W4=tf.get_variable("weight4" , shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
            b4=tf.Variable(tf.random_normal([10]),name="bias4" )

            self.H = tf.matmul(layer2,W4)+b4

            self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.H , labels=self.Y))

            self.train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(self.cost)

            self.predict = tf.argmax(self.H,1)
            self.correct=tf.equal(self.predict , tf.argmax(self.Y,1)) 

            self.accuracy=tf.reduce_sum(tf.cast(self.correct , dtype=tf.float32))

    
    def train_model(self,x,y):

        

        n=0
        n1=100
        n2=100
        for step in range(10):
            
            n=0
            n1=100
            n2=100

            for i in range(420):
                batch_x = x[n:n2]
                batch_y = y[n:n2]
                _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.X:batch_x,self.Y:batch_y 
                                                                            ,self.keep_rate:0.5})
                n=n+n1
                n2=n2+n1
            print("cost : {}".format(cost_val))

    def H_value(self,x):
        

        n=0
        n1=100
        n2=100
        H_val_arraysum=[]
        H_val_arraysumDF=pd.DataFrame()
        #H_val_arraysumDF=pd.DataFrame
        ran=420
        for i in range(ran):
            batch_x = x[n:n2]
            H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))
            H_val_arraysum.extend(H_val) # 리스트방법
            
            
            #H_val_arraysumDF=pd.DataFrame(H_val)
            #if i>=2:
            #    H_val_arraysumDFresult=pd.merge((H_val_arraysumDF(i)),(H_val_arraysumDF(i+1)),how="outer")
            #c=np.vstack((H_val,H_val))
            
            #H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:1 }))
            n=n+n1
            n2=n2+n1
        #print("H : {}".format(H_val))
        
        #return self.sess.run(self.H, feed_dict={self.X:x , self.keep_rate:1 })
        return H_val_arraysum
    
    
    def H_value_sum(self,x,y):
        

        n=0
        n1=100
        n2=100
        H_val_arraysum=[]
        H_zero = np.zeros((100,10),dtype=np.float32)
        ran=420
        for i in range(ran):
            ### 모델 10개
            model_list = ["m0","m1"]#,"m2","m3","m4","m5","m6","m7","m8","m9"]
            
            for k in model_list:
                batch_x = x[n:n2]
                batch_y = y[n:n2]
                H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))

                H_zero=H_zero+H_val
            
            H_modelsum=H_zero/10
            
            _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.H:H_modelsum , self.Y:batch_y , self.keep_rate:0.5})
            ### 모델 10개 끝
            
            ##시험삼아
#             self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.H , labels=self.Y))

#             self.train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(self.cost)

#             self.predict = tf.argmax(self.H,1)
#             self.correct=tf.equal(self.predict , tf.argmax(self.Y,1)) 

#             self.accuracy=tf.reduce_sum(tf.cast(self.correct , dtype=tf.float32))
            
            ######    시험삼아
            n=n+n1
            n2=n2+n1
      
        print("cost : {}".format(cost_val))

        ##모델 10개
#         def model(self)
#         model_list = ["m0","m1"]#,"m2","m3","m4","m5","m6","m7","m8","m9"]
            
#             for k in model_list:
#                 batch_x = x[n:n2]
#                 batch_y = y[n:n2]
#                 H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))

#                 H_zero=H_zero+H_val
            
#             H_modelsum=H_zero/10
#         ###모델 10개    
           # _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.H:H_modelsum , self.Y:batch_y , self.keep_rate:0.5})
            
#     def predict(self):
#     def correct(self):
#     def accuracy(self):

sess=tf.Session()

#for model_roof in range(5):

# model_list = ["m0","m1","m2","m3","m4","m5","m6","m7","m8","m9"]
# model_roof=0
# for k in model_list:
    
k=CNNModel(sess,"model1")   #객체생성
k1=k.build_graph()             #모델생성   m1이 빌드 그래프를 가지고 있다 

sess.run(tf.global_variables_initializer())
#m1.train_model(x_data,y_data)

#print(m1.H_value(x_data)) # [[-3.2657056   0.75052345 -1.8666681    2.8347716   1.7404464   3.1127691 ][][]...[]]100개



k1.H_value_sum(x_data,y_data)



# 42000










KeyboardInterrupt: 

In [113]:
# import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings


### x_data , y_data , dataframe 형태
tf.reset_default_graph()

train = pd.read_csv("./data/mnist2/train.csv")
test = pd.read_csv("./data/mnist2/test.csv")


x_data=train.drop("label", axis=1, inplace=False)
train2=train["label"]

 
df1=pd.DataFrame()
arr = np.zeros((10,10), dtype=np.int32)
df1=arr
df2=pd.DataFrame()
for n in range(0,10):
    df1[n,n]=1
    df2[n]=np.arange(10)

tr2=int(len(train2))
for n in range(0,tr2):
    df2[n]=df1[train2[n]]
y_data=pd.DataFrame()
y_data=df2.transpose()


x_data=x_data.values
y_data=y_data.values
### x_data , y_data , dataframe 형태 넘파이 어레이가 공통적으로 쓰임

class CNNModel: 
    
    def __init__(self,sess,name):     

        self.sess=sess
        self.name=name 
        
    def build_graph(self):
        with tf.variable_scope(self.name):
            self.X=tf.placeholder(shape=[None,784], dtype=tf.float32)
            self.Y=tf.placeholder(shape=[None,10], dtype=tf.float32)
            self.keep_rate=tf.placeholder(dtype=tf.float32) 
            
            X_img = tf.reshape(self.X, shape=[-1,28,28,1])

            W1=tf.Variable(tf.random_normal([5,5,1,32],stddev=0.01))

            L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1], padding="SAME")

            L1 = tf.nn.relu(L1)
            p1 = tf.nn.max_pool(L1 , ksize=[1,2,2,1] , strides=[1,2,2,1], padding="SAME")

            L2=tf.layers.conv2d(inputs=p1, filters=64, kernel_size=[5,5] , padding="SAME", strides=1 ,activation=tf.nn.relu)

            L2=tf.layers.max_pooling2d(inputs=L2 , pool_size=[2,2], padding="SAME" , strides=2)

            L2=tf.reshape(L2,shape=[-1,7*7*64])

            W2=tf.get_variable("weight2" , shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
            b2=tf.Variable(tf.random_normal([256]),name="bias2" )

            _layer1 = tf.nn.relu(tf.matmul(L2,W2)+b2 )
            layer1 = tf.nn.dropout(_layer1 , keep_prob=self.keep_rate)


            W3=tf.get_variable("weight3" , shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
            b3=tf.Variable(tf.random_normal([256]),name="bias3" )

            _layer2 = tf.nn.relu(tf.matmul(layer1,W3)+b3 )
            layer2 = tf.nn.dropout(_layer2 , keep_prob=self.keep_rate)


            W4=tf.get_variable("weight4" , shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
            b4=tf.Variable(tf.random_normal([10]),name="bias4" )

            self.H = tf.matmul(layer2,W4)+b4

            self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.H , labels=self.Y))

            self.train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(self.cost)

            self.predict = tf.argmax(self.H,1)
            self.correct=tf.equal(self.predict , tf.argmax(self.Y,1)) 

            self.accuracy=tf.reduce_sum(tf.cast(self.correct , dtype=tf.float32))

    
    def train_model(self,x,y):

        

        n=0
        n1=100
        n2=100
        for step in range(10):
            
            n=0
            n1=100
            n2=100

            for i in range(420):
                batch_x = x[n:n2]
                batch_y = y[n:n2]
                _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.X:batch_x,self.Y:batch_y 
                                                                            ,self.keep_rate:0.5})
                n=n+n1
                n2=n2+n1
            print("cost : {}".format(cost_val))

    def H_value(self,x):
        

        n=0
        n1=100
        n2=100
        H_val_arraysum=[]
        H_val_arraysumDF=pd.DataFrame()
        #H_val_arraysumDF=pd.DataFrame
        ran=1
        for i in range(ran):
            batch_x = x[n:n2]
            H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))
            H_val_arraysum.extend(H_val) # 리스트방법
            
            
            #H_val_arraysumDF=pd.DataFrame(H_val)
            #if i>=2:
            #    H_val_arraysumDFresult=pd.merge((H_val_arraysumDF(i)),(H_val_arraysumDF(i+1)),how="outer")
            #c=np.vstack((H_val,H_val))
            
            #H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:1 }))
            n=n+n1
            n2=n2+n1
        #print("H : {}".format(H_val))
        
        #return self.sess.run(self.H, feed_dict={self.X:x , self.keep_rate:1 })
        return H_val_arraysum
    
    
    def H_value_sum(self,x,y):
        
        n=0
        n1=100
        n2=100
        H_val_arraysum=[]
        H_zero = np.zeros((100,10),dtype=np.float32)
        ran=420
        for i in range(ran):
            ### 모델 10개
            model_list = ["m0","m1"]#,"m2","m3","m4","m5","m6","m7","m8","m9"]
            
            for k in model_list:
                batch_x = x[n:n2]
                batch_y = y[n:n2]
                H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))

                H_zero=H_zero+H_val
            
            self.H=H_zero/10
            print(self.H)
#             _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.H:H_modelsum , self.Y:batch_y , self.keep_rate:0.5})
#             ### 모델 10개 끝
            
#             ##시험삼아 시작
#             self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.H , labels=self.Y))

#             self.train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(self.cost)

#             self.predict = tf.argmax(self.H,1)
#             self.correct=tf.equal(self.predict , tf.argmax(self.Y,1)) 

#             self.accuracy=tf.reduce_sum(tf.cast(self.correct , dtype=tf.float32))
            
#             ######    시험삼아 끝
            n=n+n1
            n2=n2+n1
            return(self.H)
            #print("cost : {}".format(cost_val))

        
#         def model_sum(self,):
            
            
#                  ### 모델 10개
#             model_list = ["m0","m1"]#,"m2","m3","m4","m5","m6","m7","m8","m9"]
            
#             for k in model_list:
#                 batch_x = x[n:n2]
#                 batch_y = y[n:n2]
#                 H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))

#                 H_zero=H_zero+H_val
            
#             H_modelsum=H_zero/10
            
#             _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.H:H_modelsum , self.Y:batch_y , self.keep_rate:0.5})
#             ### 모델 10개 끝
            
    def model_h_value(self,h,y):


        batch_y=y[0:100]
        
        self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.H , labels=self.Y))

        self.train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(self.cost)

        self.predict = tf.argmax(self.H,1)
        self.correct=tf.equal(self.predict , tf.argmax(self.Y,1)) 

        self.accuracy=tf.reduce_sum(tf.cast(self.correct , dtype=tf.float32))
        
        cost_val=self.sess.run(self.cost, feed_dict={self.H:h,self.Y:batch_y 
                                                                        ,self.keep_rate:0.5})

        
        print("cost : {}".format(cost_val))


sess=tf.Session()


###모델 10개
# model_list = ["m0","m1","m2","m3","m4","m5","m6","m7","m8","m9"]
# model_roof=0
# for k in model_list:
###모델 10개 끝
k=CNNModel(sess,"model1")   #객체생성
k.build_graph()             #모델생성   m1이 빌드 그래프를 가지고 있다 

sess.run(tf.global_variables_initializer())
#m1.train_model(x_data,y_data)

#print(m1.H_value(x_data)) # [[-3.2657056   0.75052345 -1.8666681    2.8347716   1.7404464   3.1127691 ][][]...[]]100개



self_H=k.H_value_sum(x_data,y_data)


k2=CNNModel(sess,"model2")
k2.model_h_value(self_H,y_data)


# 42000




[[-5.15280664e-01 -2.59530604e-01 -2.43090510e-01  4.81846720e-01
  -9.74919200e-02  4.64347303e-01 -8.63171399e-01 -9.79597569e-02
  -3.60020131e-01  1.37562490e+00]
 [-3.20999837e+00 -2.73733616e+00  4.44712698e-01  1.12117648e+00
   2.12275457e+00  3.06936502e+00 -1.28460288e+00  3.53754133e-01
   9.98587757e-02  1.10312030e-01]
 [-2.51605898e-01 -2.96605714e-02 -6.95361674e-01  6.49563551e-01
   3.97425950e-01 -7.55857110e-01  4.07875717e-01 -2.37087637e-01
   5.99240303e-01  1.11820865e+00]
 [-1.75812149e+00 -1.12838817e+00 -5.22349954e-01 -8.17737401e-01
   5.60075104e-01 -5.38276315e-01 -8.07640731e-01 -8.68142009e-01
  -1.51702428e+00  1.45308092e-01]
 [-1.73375928e+00  2.68738806e-01  1.12922117e-01  1.04283404e+00
   1.02029383e+00  5.91213882e-01 -1.06042540e+00  8.92904878e-01
   6.34238273e-02 -1.10842013e+00]
 [-1.26369989e+00 -1.20135200e+00  1.80810779e-01 -7.69256294e-01
   1.20329630e+00  1.17417932e+00 -1.89351213e+00 -5.90192020e-01
  -3.47127616e-01  1.60227966e+00

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'model1/Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>", "<tf.Variable 'model1/conv2d/kernel:0' shape=(5, 5, 32, 64) dtype=float32_ref>", "<tf.Variable 'model1/conv2d/bias:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'model1/weight2:0' shape=(3136, 256) dtype=float32_ref>", "<tf.Variable 'model1/bias2:0' shape=(256,) dtype=float32_ref>", "<tf.Variable 'model1/weight3:0' shape=(256, 256) dtype=float32_ref>", "<tf.Variable 'model1/bias3:0' shape=(256,) dtype=float32_ref>", "<tf.Variable 'model1/weight4:0' shape=(256, 10) dtype=float32_ref>", "<tf.Variable 'model1/bias4:0' shape=(10,) dtype=float32_ref>"] and loss Tensor("Mean:0", shape=(), dtype=float32).

In [1]:
# import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings

## reset tensorflow graph
tf.reset_default_graph()

class CNNModel: 
    
    def __init__(self,sess,name):     

        self.sess=sess
        self.name=name 
        
    def build_graph(self):
        with tf.variable_scope(self.name):
            self.X=tf.placeholder(shape=[None,784], dtype=tf.float32)
            self.Y=tf.placeholder(shape=[None,10], dtype=tf.float32)
            self.keep_rate=tf.placeholder(dtype=tf.float32) 
            
            X_img = tf.reshape(self.X, shape=[-1,28,28,1])

            W1=tf.Variable(tf.random_normal([5,5,1,32],stddev=0.01))

            L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1], padding="SAME")

            L1 = tf.nn.relu(L1)
            p1 = tf.nn.max_pool(L1 , ksize=[1,2,2,1] , strides=[1,2,2,1], padding="SAME")

            L2=tf.layers.conv2d(inputs=p1, filters=64, kernel_size=[5,5] , padding="SAME", strides=1 ,activation=tf.nn.relu)

            L2=tf.layers.max_pooling2d(inputs=L2 , pool_size=[2,2], padding="SAME" , strides=2)

            L2=tf.reshape(L2,shape=[-1,7*7*64])

            W2=tf.get_variable("weight2" , shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
            b2=tf.Variable(tf.random_normal([256]),name="bias2" )

            _layer1 = tf.nn.relu(tf.matmul(L2,W2)+b2 )
            layer1 = tf.nn.dropout(_layer1 , keep_prob=self.keep_rate)


            W3=tf.get_variable("weight3" , shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
            b3=tf.Variable(tf.random_normal([256]),name="bias3" )

            _layer2 = tf.nn.relu(tf.matmul(layer1,W3)+b3 )
            layer2 = tf.nn.dropout(_layer2 , keep_prob=self.keep_rate)


            W4=tf.get_variable("weight4" , shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
            b4=tf.Variable(tf.random_normal([10]),name="bias4" )

            self.H = tf.matmul(layer2,W4)+b4

            self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.H , labels=self.Y))

            self.train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(self.cost)

            self.predict = tf.argmax(self.H,1)
            self.correct=tf.equal(self.predict , tf.argmax(self.Y,1)) 

            self.accuracy=tf.reduce_sum(tf.cast(self.correct , dtype=tf.float32))

    
    def train_model(self,x,y):
        
        _,cost_val=self.sess.run([self.train,self.cost], feed_dict={self.X:batch_x,self.Y:batch_y 
                                                                            ,self.keep_rate:0.5})

        n=0
        n1=100
        n2=100
        for step in range(10):
            
            n=0
            n1=100
            n2=100

            for i in range(420):
                batch_x = x[n:n2]
                batch_y = y[n:n2]
                n=n+n1
                n2=n2+n1
            print("cost : {}".format(cost_val))
            
    
    def H_value_sum(self,x,y):
        
        n=0
        n1=100
        n2=100
        H_zero = np.zeros((100,10),dtype=np.float32)
        ran=420
        for i in range(ran):
            ### 모델 10개
            model_list = ["m0","m1"]#,"m2","m3","m4","m5","m6","m7","m8","m9"]
            
            for k in model_list:
                batch_x = x[n:n2]
                batch_y = y[n:n2]
                H_val=(self.sess.run(self.H, feed_dict={self.X:batch_x , self.keep_rate:0.5 }))

                H_zero=H_zero+H_val
            
            self.H=H_zero/10
            print(self.H)

            n=n+n1
            n2=n2+n1
        return(self.H)

    ### x_data , y_data , dataframe 형태
tf.reset_default_graph()

train = pd.read_csv("./data/mnist2/train.csv")
test = pd.read_csv("./data/mnist2/test.csv")


x_data=train.drop("label", axis=1, inplace=False)
train2=train["label"]

 
df1=pd.DataFrame()
arr = np.zeros((10,10), dtype=np.int32)
df1=arr
df2=pd.DataFrame()
for n in range(0,10):
    df1[n,n]=1
    df2[n]=np.arange(10)

tr2=int(len(train2))
for n in range(0,tr2):
    df2[n]=df1[train2[n]]
y_data=pd.DataFrame()
y_data=df2.transpose()


x_data=x_data.values
y_data=y_data.values
### x_data , y_data , dataframe 형태 넘파이 어레이가 공통적으로 쓰임


count_of_models = 10
models = [CNNModel(sess,"model"+ str(x)) for x in range(count_of_models)]
for num in range(count_of_models):
    models[num].build_graph()
    print(models[num])
    models=models.build_graph()

    start_roof=0
    increase_roof=100
    finish_roof=100

    H_zero = np.zeros((100,10),dtype=np.float32)
    n=0
    n1=100
    n2=100
    for step in range(10):

        n=0
        n1=100
        n2=100

        for i in range(420):
            batch_x = x[n:n2]
            batch_y = y[n:n2]
            models[num].train_model
            n=n+n1
            n2=n2+n1

        print("cost : {}".format(cost_val))


NameError: name 'sess' is not defined

In [117]:
print(models[0])

(100, 10)

In [69]:
h1[41000]

array([-2.4025044 ,  3.7187655 , -1.9722697 , -0.46925777, -0.29228485,
        7.928941  ,  9.954054  , -0.65744174,  6.9954424 , -0.76876533],
      dtype=float32)

In [55]:
#


# list 방법

print(h1[0]+h1[1])
print((h1[0]+h1[1])/2)


[ -6.6541653   -3.9277194    6.4324713   -7.7255635   -0.88025427
  19.678463   -13.832499     0.17511225   4.946245   -17.161434  ]
[-3.3270826  -1.9638597   3.2162356  -3.8627818  -0.44012713  9.8392315
 -6.9162493   0.08755612  2.4731226  -8.580717  ]


In [ ]:
import tensorflow as tf

class Student:   #클레스 
    # 언더바 2개가 뭘 의미한댔지?
    def __init__(self,name,kor,eng,math):       # self-> 자바에서 this 함수이기때문에 arg 리스트가 나옴

        self.name=name  # 객체가 존재하는한 계속해서 남아있는 객체  자기가 가지고 있는
        self.kor=kor
        self.eng=eng
        self.math=math
    
    def calc_avg(self):
        return (self.kor+self.eng+self.math)/3
stu1=Student("홍길동",10,20,30)
print(stu1.calc_avg())

stu2=Student("김길동",10,20,30)
print(stu2.calc_avg())


def __init__(self,sess,name,train1,df4)

In [58]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings

lst1 = [
    [1,2],
    [3,4]
]
a=np.array(lst1)
df=pd.DataFrame(a)

print(df)

TypeError: 'df' is an invalid keyword argument for this function

In [84]:
import numpy as np
 
lst1 = [
    [1,2],
    [3,4]
]
a=np.array(lst1)
b=np.array(lst1)
print(lst1)
display(a)
display(b)

c=np.vstack((a,b))
print(c)
display(c)

c=np.vstack((c,b))
c=np.vstack((c,b))
H_val_sum=np.vstack((a,b))
display(c)
display(H_val_sum)

e=a+b

print(e/10)

model_list = ["m0","m1","m2","m3","m4","m5","m6","m7","m8","m9"]

for k in model_list:
    print(k)

    
H_zero = np.zeros((100,10),dtype=np.float32)
print(H_zero)                  


[[1, 2], [3, 4]]


array([[1, 2],
       [3, 4]])

array([[1, 2],
       [3, 4]])

[[1 2]
 [3 4]
 [1 2]
 [3 4]]


array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

[[0.2 0.4]
 [0.6 0.8]]
m0
m1
m2
m3
m4
m5
m6
m7
m8
m9
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 